<a href="https://colab.research.google.com/github/mab2004/Multimodal-Housing-Price-Prediction/blob/main/Multimodal_Housing_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup, Installation, and Data Download

In [ ]:
# Install necessary libraries (re-run to ensure all are installed)
!pip install tensorflow scikit-learn numpy pandas matplotlib imutils opencv-python

# Install git if not present and clone the Houses Dataset repository
!apt install git -y
!git clone https://github.com/emanhamed/Houses-dataset.git

# Define the base path for the cloned dataset
BASE_PATH = "Houses-dataset/Houses Dataset"

## Data Loading, Preprocessing, and Scaling

In [4]:
import numpy as np
import pandas as pd
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import get_file

# --- A. Load Tabular Data (HousesInfo.txt) ---
TABULAR_FILE = os.path.join(BASE_PATH, "HousesInfo.txt")
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv(TABULAR_FILE, sep=" ", header=None, names=cols)

# --- B. Image Processing Setup ---
image_paths = sorted(list(paths.list_images(BASE_PATH)))
IMAGE_SIZE = (64, 64) # Target size for each of the 4 images

# Function to extract the unique house ID from the image filename (e.g., '1_bedroom.jpg' -> 1)
def get_house_id(path):
    return int(path.split(os.path.sep)[-1].split("_")[0])

# Match image paths to the DataFrame indices
df['house_id'] = range(1, len(df) + 1)
house_ids = [get_house_id(p) for p in image_paths]

images = []
targets = []
# Define the tile positions for merging the 4 images into one 128x128 image
TILE_MAP = {"frontal": (0, 0), "bedroom": (0, 1), "bathroom": (1, 0), "kitchen": (1, 1)}

print("[INFO] Loading and preprocessing images...")

for i, row in df.iterrows():
    house_id = row['house_id']
    price = row['price']

    # Filter for the four images corresponding to the current house_id
    house_image_paths = [p for p in image_paths if get_house_id(p) == house_id]

    if len(house_image_paths) == 4:
        tiled_image = np.zeros((IMAGE_SIZE[0] * 2, IMAGE_SIZE[1] * 2, 3), dtype="uint8")

        for p in house_image_paths:
            img_type = os.path.basename(p).split("_")[1].split(".")[0]
            if img_type in TILE_MAP:
                image = cv2.imread(p)
                image = cv2.resize(image, IMAGE_SIZE)

                # Place the image in the correct 2x2 tile position
                (r, c) = TILE_MAP[img_type]
                tiled_image[r*IMAGE_SIZE[0]:(r+1)*IMAGE_SIZE[0], c*IMAGE_SIZE[1]:(c+1)*IMAGE_SIZE[1]] = image

        images.append(tiled_image)
        targets.append(price)

images = np.array(images) / 255.0 # Normalize pixel values to [0, 1]
targets = np.array(targets)
tabular_data = df.drop('price', axis=1)[cols[:-1]].values # Get tabular features

print(f"[INFO] Loaded {len(images)} samples (houses).")

# --- C. Scaling and Splitting ---

# Scale tabular features (Area and Price are large)
# NOTE: Price scaling is often applied for NN output stability, but here we scale inputs only for interpretability
scaler = MinMaxScaler()
tabular_data = scaler.fit_transform(tabular_data)

# Split the data into training and testing sets (75% train, 25% test)
split = train_test_split(tabular_data, images, targets, test_size=0.25, random_state=42)
(X_tabular_train, X_tabular_test, X_image_train, X_image_test, y_train, y_test) = split

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


[INFO] Loading and preprocessing images...
[INFO] Loaded 535 samples (houses).


## Multimodal Model Architecture (CNN + MLP Fusion)


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam

# --- A. Define Inputs ---
image_input_shape = X_image_train.shape[1:]
tabular_input_shape = X_tabular_train.shape[1:]

# --- B. CNN Branch for Images ---
image_input = Input(shape=image_input_shape, name='image_input')
x = Conv2D(16, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)

# --- C. Tabular Branch (MLP) ---
tabular_input = Input(shape=tabular_input_shape, name='tabular_input')
y = Dense(16, activation='relu')(tabular_input)
y = Dense(8, activation='relu')(y)

# --- D. Feature Fusion and Output ---
combined_features = concatenate([x, y])
z = Dense(32, activation='relu')(combined_features)
output = Dense(1, activation='linear')(z) # Linear activation for regression (price prediction)

# Create and compile the final model
model = Model(inputs=[image_input, tabular_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mean_squared_error')

print("[INFO] Model Architecture:")
model.summary()

[INFO] Model Architecture:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 126, 126,  │        448 │ image_input[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 63, 63,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 61, 61,    │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 30, 30,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabular_input       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 28800)     │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │         80 │ tabular_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │  1,843,264 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8)         │        136 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 72)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,336 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         33 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,850,937 (7.06 MB)

 Trainable params: 1,850,937 (7.06 MB)

 Non-trainable params: 0 (0.00 B)

## Training and Evaluation

In [7]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Training the Model ---
print("\n[INFO] Training model...")
history = model.fit(
    {'image_input': X_image_train, 'tabular_input': X_tabular_train},
    y_train,
    epochs=25,
    batch_size=8,
    validation_split=0.1,
    verbose=2 # Show one line per epoch
)

# --- Evaluation ---
print("\n[INFO] Evaluating model...")
predictions = model.predict([X_image_test, X_tabular_test])

# Calculate MAE and RMSE
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Format the output clearly
print("\n" + "="*40)
print(" Multimodal Housing Price Prediction Results")
print("="*40)
print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:,.2f}")
print("="*40)


[INFO] Training model...
Epoch 1/25
45/45 - 2s - 42ms/step - loss: 226294611968.0000 - val_loss: 147577880576.0000
Epoch 2/25
45/45 - 2s - 41ms/step - loss: 227620618240.0000 - val_loss: 151106338816.0000
Epoch 3/25
45/45 - 2s - 41ms/step - loss: 224215564288.0000 - val_loss: 143819948032.0000
Epoch 4/25
45/45 - 2s - 42ms/step - loss: 222972641280.0000 - val_loss: 146888048640.0000
Epoch 5/25
45/45 - 2s - 41ms/step - loss: 228330553344.0000 - val_loss: 155635449856.0000
Epoch 6/25
45/45 - 2s - 42ms/step - loss: 223803310080.0000 - val_loss: 150280863744.0000
Epoch 7/25
45/45 - 2s - 42ms/step - loss: 223021858816.0000 - val_loss: 141790347264.0000
Epoch 8/25
45/45 - 2s - 42ms/step - loss: 222885642240.0000 - val_loss: 159733972992.0000
Epoch 9/25
45/45 - 2s - 41ms/step - loss: 224355631104.0000 - val_loss: 141026639872.0000
Epoch 10/25
45/45 - 2s - 44ms/step - loss: 219215921152.0000 - val_loss: 151284842496.0000
Epoch 11/25
45/45 - 2s - 44ms/step - loss: 218840203264.0000 - val_loss: 